In [1]:
from dotenv import load_dotenv
from helpers.graphql_provider import SafesProvider
from helpers.hoprd_api import HoprdAPI
from helpers.utils import Utils
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.patches import Rectangle
import numpy as np
from ipywidgets import interactive, VBox, Layout, HBox, IntSlider, FloatSlider, IntRangeSlider

from models.economic_model import EconomicModel, Equations, Parameters, BudgetParameters, Equation
from models.peer import Peer
from models.subgraph_entry import SubgraphEntry
from models.tolopogy_entry import TopologyEntry

# %matplotlib inline
%config InlineBackend.figure_format = 'retina'

_ = load_dotenv()

### Instanciate an API handler

In [2]:
api = HoprdAPI(*Utils.apiHostAndKey('API_HOST', "API_KEY"))
safes_balances_subgraph_url = Utils.envvar('SAFES_BALANCES_SUBGRAPH_URL', str)

async def get_subgraph_data():
    key = "registeredNodesInNetworkRegistry"
    query_id = "SUBGRAPH_SAFES_BALANCES_QUERY_ID"
    safes_provider = SafesProvider(Utils.buildSubgraphURL(query_id))

    results = list[SubgraphEntry]()
    for safe in await safes_provider.get():
        results.extend([SubgraphEntry.fromDict(node) for node in safe[key]])

    return results

async def get_topology_data(api: HoprdAPI):
    channels = await api.all_channels(False)

    results = await Utils.aggregatePeerBalanceInChannels(channels.all)
    return [TopologyEntry.fromDict(*arg) for arg in results.items()]

async def get_peers(api: HoprdAPI):
    fields = ["peer_id", "peer_address", "reported_version"]

    node_result = await api.peers(params=fields, quality=0.5)

    return { Peer(*[item[f] for f in fields]) for item in node_result }

def get_eligibles_peers(topology: list[TopologyEntry], peers: list[Peer], safes: list[SubgraphEntry]):
    eligibles = Utils.mergeTopoPeersSafes(topology, peers, safes)

    addresses_to_exclude = [peer.address for peer in eligibles if peer.version_is_old(min_version)]
    Utils.exclude(eligibles, addresses_to_exclude)

    addresses_to_exclude = [peer.address for peer in eligibles if peer.safe_allowance < 0]
    Utils.exclude(eligibles, addresses_to_exclude)

    Utils.allowManyNodePerSafe(eligibles)

    return eligibles

In [3]:
safes = await get_subgraph_data()
print(f"Subgraph size: {len(safes)}")

topology = await get_topology_data(api)
print(f"Topology size: {len(topology)}")

peers = await get_peers(api)
print(f"Number of peers: {len(peers)}")

Subgraph size: 659
Topology size: 437
Number of peers: 354


In [4]:
min_version = "2.0.7"
budget_dollar = 7500
token_price = 0.05
min_stake_wxhopr = 10e3
thrs_stake_wxhopr = 75e3

### Define and apply economic model

In [8]:
def get_reward_probability(token_price: float, budget_dollars: int, limits: list[int]):
    equations = Equations(
        Equation("a * x", "l <= x <= c"), 
        Equation("a * c + (x - c) ** (1 / b)", "x > c")
    )

    parameters = Parameters(1, 1.4, limits[1], limits[0])
    budget_params = BudgetParameters(budget_dollars/token_price, 2628000, 1, 365, 0.03, 1.0)
    economic_model = EconomicModel(equations, parameters, budget_params)
    
    eligibles = get_eligibles_peers(topology, peers, safes)
    
    for peer in eligibles:
        peer.economic_model = economic_model
    Utils.rewardProbability(eligibles)

    return eligibles

def plot_hist(bin_count: int, token_price: float, budget_dollars: int, limits: list[int]):
    peers = get_reward_probability(token_price, budget_dollars, limits)
    stakes = [peer.total_balance for peer in peers]

    separators = [0, 100e3, 500e3]
    splits = []
    bins = []

    for low, high in zip(separators, separators[1:]):
        splits.append([s for s in stakes if low < s < high])
        bins.append(list(range(int(low), int(high), int((high-low)/bin_count))))

    _, axes = plt.subplots(1, len(splits), figsize=(8*len(splits), 3.5), sharex=False, sharey=False)

    axes = axes.flatten() if isinstance(axes, np.ndarray) else [axes]

    for ax, split, bin in zip(axes, splits, bins):
        bin_size = bin[1] - bin[0]
        bin.append(bin[-1] + bin_size)
        min_bin = bin[0] - bin_size
        max_bin = bin[-1] + bin_size

        aprs = [peer.apr_percentage for peer in peers]
        avg_apr = sum(aprs) / len(aprs)

        _, edges, counts = ax.hist(split, bins=bin, facecolor="#000050", edgecolor='black', linewidth=1.2, rwidth=0.85)
        [ax.axvline(lim, color='#000050', linestyle='dashed', alpha=0.75) for lim in limits]

        ax.add_patch(Rectangle((0, 0), limits[0], 100, color='red', alpha=0.4, zorder=0))
        ax.add_patch(Rectangle((limits[0], 0), limits[1]-limits[0], 100, color='#ffffa0', alpha=0.4, zorder=0))

        ax.bar_label(counts)
        ax.set_xlim(min_bin, max_bin)
        ax.set_xlabel("Stake (wxHOPR)")
        ax.yaxis.set_major_locator(MaxNLocator(integer=True))
        texts = [
            ax.text(0.95, 0.95, f"Bin size: {edges[1]-edges[0]:.0f} wxHOPR", transform=ax.transAxes, ha='right', va='top'),
            ax.text(0.95, 0.85, f"Distributed tokens: {peers[0].economic_model.budget.budget:_.0f} wxHOPR", transform=ax.transAxes, ha='right', va='top'),
            ax.text(0.95, 0.75, f"APR (avg.): {avg_apr:.2f} %", transform=ax.transAxes, ha='right', va='top'),
        ]
        [t.set_bbox(dict(facecolor='#b4f0ff', alpha=0.5, edgecolor='white')) for t in texts]

    _ = axes[0].set_ylabel("Number of peers")

def plot_hist_interactive():
    style = {'description_width': 'initial'}

    bin_slider = IntSlider(min=5, max=20, step=1, value=10, description="Bins count", style=style)
    budget_dollars_slider = IntSlider(min=0, max=20000, step=500, value=budget_dollar, description="Budget ($)", style=style)
    token_price_slider = FloatSlider(min=0.01, max=0.20, step=0.001, value=token_price, description="Token price ($)",readout_format='.3f', style=style)
    limits_slider = IntRangeSlider(min=0, max=200e3, step=5e3, description="Limits", value=[min_stake_wxhopr, thrs_stake_wxhopr], style=style)

    widgets = interactive(plot_hist, bin_count=bin_slider, token_price=token_price_slider, budget_dollars=budget_dollars_slider, limits=limits_slider)
    output = widgets.children[-1]
    controls = HBox(widgets.children[:-1], layout = Layout(flex_flow='row wrap'))
    display(VBox([controls, output]))


In [9]:
plot_hist_interactive()